## Import Libraries

In [1]:
import pandas_datareader.data as pdr
import datetime
import numpy as np
import unittest
import pandas as pd

# seems to be a problem with pandas_datareader
# use this fix to get data from Yahoo! Finance
import fix_yahoo_finance as yf
yf.pdr_override()

/Users/Brent/anaconda/envs/stocks/lib/python3.6/site-packages/fix_yahoo_finance/__init__.py:43: DeprecationWarning: 
    Auto-overriding of pandas_datareader's get_data_yahoo() is deprecated and will be removed in future versions.
    Use pdr_override() to explicitly override it.
  DeprecationWarning)


## Helper Functions

In [2]:
def run_unit_tests(testClass):
    suite = unittest.TestLoader().loadTestsFromTestCase(testClass)
    unittest.TextTestRunner().run(suite)

## Data
Create a class that downloads the stock data and sets up the input/output data

In [8]:
class CreateDataSet:
    
    def __init__(self,symbol,number_of_days=10):
        self.symbol = symbol
        self.number_of_days = number_of_days
        self.train_start_date = None
        self.train_end_date = None
        self.test_start_date = None
        self.test_end_date = None
        self.validation_start_date = None
        self.validation_end_date = None
        self.yahoo_data = None
        self.pandas_df = None
        self.training_df = None
        self.test_df = None
        self.validation_df = None
        self.training_x = None
        self.training_y = None
        self.test_x = None
        self.test_y = None
        self.validation_x = None
        self.validation_y = None

        
    def set_training_dates(self,start_date,end_date):
        self.__check_date_string_format(start_date)
        self.__check_date_string_format(end_date)
        self.__check_start_date_before_end_date(start_date,end_date)
        self.train_start_date = start_date
        self.train_end_date = end_date
        
    def set_test_dates(self,start_date,end_date):
        self.__check_date_string_format(start_date)
        self.__check_date_string_format(end_date)
        self.__check_start_date_before_end_date(start_date,end_date)
        self.test_start_date = start_date
        self.test_end_date = end_date
        
    def set_validation_dates(self,start_date,end_date):
        self.__check_date_string_format(start_date)
        self.__check_date_string_format(end_date)
        self.__check_start_date_before_end_date(start_date,end_date)
        self.validation_start_date = start_date
        self.validation_end_date = end_date
        
    def download_data(self):
        assert self.train_start_date != None, "Training start date not set"
        assert self.train_end_date != None, "Training end date not set"
        assert self.test_start_date != None, "Test start date not set"
        assert self.test_end_date != None, "Test end date not set"
        assert self.validation_start_date != None, "Validation start date not set"
        assert self.validation_end_date != None, "Validation end date not set"
        
        start_date, end_date = self.__calculate_download_start_end_dates()

        self.yahoo_data = self.__download_data_yahoo(self.symbol,start_date,end_date)
        
        assert self.yahoo_data.shape != (0,0), "No data was downloaded"
        
    def create_dataset(self):
        
        if self.yahoo_data == None:
            self.download_data()
        
        prices = self.yahoo_data['Adj Close']
        
        # fill in missing data
        prices.fillna(method='ffill', inplace=True)
        prices.fillna(method='bfill', inplace=True)
        
        # input features named X0, X1, X2, ...
        # output feature named Y (this is what we are predicting, tomorrows stock price)
        x_features = [ 'X%d' % (i) for i in range(self.number_of_days)]
        y_feature = ['Y']
        features = x_features + y_feature
        
        df = pd.DataFrame(index=prices.index,columns=features)
        df['Y'] = prices.shift(-1)
        
        for i in range(self.number_of_days):
            x_feature = "X%d" % (i)
            df[x_feature] = prices.shift(i)
            
        # trim the dataset to only use dates with enough data
        #
        # example: if number_of_days=10, then first date with enough
        # information will be day 10 (i.e. array index 9)
        #
        # last date will be next to last day in dataset since predicting tommorrows price
        df = df[self.number_of_days-1:-1]
        
        # there shouldn't be any NaN values
        assert df.isnull().values.any() == False, "Dataset contains unexpected NaN values"
                    
        self.pandas_df = df
        
        # split dataset into training/test/validation sets
        
        # data as pandas dataframes
        self.training_df = df[self.train_start_date:self.train_end_date]
        self.test_df = df[self.test_start_date:self.test_end_date]
        self.validation_df = df[self.validation_start_date:self.validation_end_date]
        
        # data as numpy arrays
        self.training_x = self.training_df[x_features].values
        self.training_y = self.training_df[y_feature].values
        self.test_x = self.test_df[x_features].values
        self.test_y = self.test_df[y_feature].values
        self.validation_x = self.validation_df[x_features].values
        self.validation_y = self.validation_df[y_feature].values        
        
    def __download_data_yahoo(self,symbol,start_date,end_date):
        data = pdr.get_data_yahoo(symbol, start=start_date, end=end_date)
        return data
    
    def __str_to_datetime(self,date_str):
        return datetime.datetime.strptime(date_str,"%Y-%m-%d")
    
    def __check_date_string_format(self,date_str):
        assert datetime.datetime.strptime(date_str,"%Y-%m-%d"), "Dates must be a string in the format YYYY-MM-DD"
        
    def __check_start_date_before_end_date(self,start_date,end_date):
        start = self.__str_to_datetime(start_date)
        end = self.__str_to_datetime(end_date)
        assert start < end, "Start date must be before the end date"
        
    def __calculate_download_start_end_dates(self):
        train_start = self.__str_to_datetime(self.train_start_date)
        test_start = self.__str_to_datetime(self.test_start_date)
        validation_start = self.__str_to_datetime(self.validation_start_date)
        train_end = self.__str_to_datetime(self.train_end_date)
        test_end = self.__str_to_datetime(self.test_end_date)
        validation_end = self.__str_to_datetime(self.validation_end_date)
        
        start_datetime = min(train_start,test_start,validation_start)
        end_datetime = max(train_end,test_end,validation_end)
        
        start_date = start_datetime.strftime("%Y-%m-%d")
        end_date = end_datetime.strftime("%Y-%m-%d")
          
        return start_date,end_date
        
    


In [10]:
# Test the create data set class
class CreateDataSetTest(unittest.TestCase):
        
    def test_set_dates(self):
        dataset = CreateDataSet('SPY')
        dataset.set_training_dates(start_date="2010-01-01",end_date="2011-01-01")
        dataset.set_test_dates(start_date="2010-01-01",end_date="2011-01-01")
        dataset.set_validation_dates(start_date="2010-01-01",end_date="2011-01-01")
        
    def test_set_dates_not_str(self):
        dataset = CreateDataSet('SPY')
        
        with self.assertRaises(Exception):
            dataset.set_training_dates(start_date=datetime.datetime(2010,1,1),end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_training_dates(start_date="2010-01-01",end_date=datetime.datetime(2011,1,1))
            
        with self.assertRaises(Exception):
            dataset.set_test_dates(start_date=datetime.datetime(2010,1,1),end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_test_dates(start_date="2010-01-01",end_date=datetime.datetime(2011,1,1))
            
        with self.assertRaises(Exception):
            dataset.set_validation_dates(start_date=datetime.datetime(2010,1,1),end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_validation_dates(start_date="2010-01-01",end_date=datetime.datetime(2011,1,1))
            
    def test_set_dates_bad_str_format(self):
        dataset = CreateDataSet('SPY')
        
        with self.assertRaises(Exception):
            dataset.set_training_dates(start_date="01-01-2011",end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_training_dates(start_date="2010-01-01",end_date="01-01-2011")
            
        with self.assertRaises(Exception):
            dataset.set_test_dates(start_date="01-01-2011",end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_test_dates(start_date="2010-01-01",end_date="01-01-2011")
            
        with self.assertRaises(Exception):
            dataset.set_validation_dates(start_date="01-01-2011",end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_validation_dates(start_date="2010-01-01",end_date="01-01-2011")
            
    def test_set_dates_start_date_after_end_date(self):
        dataset = CreateDataSet('SPY')
        
        with self.assertRaises(Exception):
            dataset.set_training_dates(start_date="2011-01-02",end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_test_dates(start_date="2011-01-02",end_date="2011-01-01")
            
        with self.assertRaises(Exception):
            dataset.set_validation_dates(start_date="2011-01-02",end_date="2011-01-01")
            
    def test_download_start_end_dates_1(self):
        dataset = CreateDataSet('SPY')
        dataset.set_training_dates(start_date="2010-01-04",end_date="2010-06-01")
        dataset.set_test_dates(start_date="2011-01-04",end_date="2011-06-01")
        dataset.set_validation_dates(start_date="2012-01-04",end_date="2012-06-01")        
        dataset.download_data()
        
        # (start_date,end_date) should be (training_start,validation_end)
        self.__verify_start_end_date(dataset.yahoo_data,expected_start="2010-01-04",expected_end="2012-06-01")
        
    def test_download_start_end_dates_2(self):
        dataset = CreateDataSet('SPY')
        dataset.set_test_dates(start_date="2010-01-04",end_date="2010-06-01")
        dataset.set_validation_dates(start_date="2011-01-04",end_date="2011-06-01")
        dataset.set_training_dates(start_date="2012-01-04",end_date="2012-06-01")        
        dataset.download_data()
        
        # (start_date,end_date) should be (test_start,training_end)
        self.__verify_start_end_date(dataset.yahoo_data,expected_start="2010-01-04",expected_end="2012-06-01")
        
    def test_download_start_end_dates_3(self):
        dataset = CreateDataSet('SPY')
        dataset.set_validation_dates(start_date="2010-01-04",end_date="2010-06-01")
        dataset.set_training_dates(start_date="2011-01-04",end_date="2011-06-01")
        dataset.set_test_dates(start_date="2012-01-04",end_date="2012-06-01")        
        dataset.download_data()
        
        # (start_date,end_date) should be (validation_start,test_end)
        self.__verify_start_end_date(dataset.yahoo_data,expected_start="2010-01-04",expected_end="2012-06-01")
        
    def __verify_start_end_date(self,dataframe,expected_start,expected_end):
        start_date, end_date = dataframe.index[0], dataframe.index[-1]
        start_date = start_date.strftime("%Y-%m-%d")
        end_date = end_date.strftime("%Y-%m-%d")
        
        self.assertEqual(start_date,expected_start)
        self.assertEqual(end_date,expected_end)
        
    def test_create_dataset(self):
        dataset = CreateDataSet('SPY')
        dataset.set_training_dates(start_date="2010-01-04",end_date="2010-06-01")
        dataset.set_test_dates(start_date="2011-01-04",end_date="2011-06-01")
        dataset.set_validation_dates(start_date="2012-01-04",end_date="2012-06-01")        
        dataset.create_dataset()
        
        # check for 10 x features and 1 y feature
        self.assertEqual(dataset.training_x.shape[1],10)
        self.assertEqual(dataset.training_y.shape[1],1)
        self.assertEqual(dataset.test_x.shape[1],10)
        self.assertEqual(dataset.test_y.shape[1],1)
        self.assertEqual(dataset.validation_x.shape[1],10)
        self.assertEqual(dataset.validation_y.shape[1],1)
        
        # check dates
        self.__check_data_in_date_range(dataset.training_df,start_date="2010-01-04",end_date="2010-06-01")
        self.__check_data_in_date_range(dataset.test_df,start_date="2011-01-04",end_date="2011-06-01")
        self.__check_data_in_date_range(dataset.validation_df,start_date="2012-01-04",end_date="2012-06-01")
        
    def __check_data_in_date_range(self,df,start_date,end_date):
        dates = df.index
        start_timestamp = pd.to_datetime(start_date)        
        end_timestamp = pd.to_datetime(end_date)
        
        for date in dates:
            self.assertTrue(date >= start_timestamp)
            self.assertTrue(date <= end_timestamp)        
    
        
        

run_unit_tests(CreateDataSetTest)

[*********************100%***********************]  1 of 1 downloaded

.

[*********************100%***********************]  1 of 1 downloaded

.

[*********************100%***********************]  1 of 1 downloaded

.

[*********************100%***********************]  1 of 1 downloaded

.....
----------------------------------------------------------------------
Ran 8 tests in 1.202s

OK


## Get the data

Download the price data from Yahoo and load it into memory

In [2]:
def download_data_yahoo(symbol,start_date,end_date):
    data = pdr.get_data_yahoo(symbol, start=start_date, end=end_date)
    return data

In [3]:
# test download_data
start = datetime.datetime(2010,1,1)
end = datetime.datetime(2011,1,1)
test_data = download_data_yahoo('SPY',start_date="2010-01-01",end_date="2011-01-01")

[*********************100%***********************]  1 of 1 downloaded

In [5]:
test_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,112.370003,113.389999,111.510002,113.330002,97.314163,118944600
2010-01-05,113.260002,113.680000,112.849998,113.629997,97.571754,111579900
2010-01-06,113.519997,113.989998,113.430000,113.709999,97.640457,116074400
2010-01-07,113.500000,114.330002,113.180000,114.190002,98.052658,131091100
2010-01-08,113.889999,114.620003,113.660004,114.570000,98.378929,126402800


## Preprocess the data
Preprocess the data into inputs and outputs.
For example, for each time step, use the past 10 days as the input, and the price on the next day as the output.  Also, do any data cleaning that is necessary like filling in missing prices

In [8]:
# TODO improvement?
# - build pandas df
# - features X0, X1, X2, ... up to number_of_days, Y
# - use shift to build features
# - trim NaNs from start

def preprocess_data(yahoo_data,number_of_days=10):
    
    prices = yahoo_data['Adj Close']
    num_prices = len(prices)
    
    # handle missing data:  fill forward, then fill backward
    prices.fillna(method='ffill', inplace=True)
    prices.fillna(method='bfill', inplace=True)
    
    # only use dates with enough information
    # start with the first date with number_of_days in the past available
    # end with the next to last date since we are predicting 1 day in the future
    #valid_data = raw_data.iloc[number_of_days-1:-1]
    #valid_prices = prices.iloc[number_of_days-1:-1]
    start_index = number_of_days - 1
    end_index = num_prices - 1
    num_samples = end_index - start_index
    
    # build the inputs and outputs
    input_x = np.empty((num_samples,number_of_days),dtype=np.float)
    output_y = np.empty((num_samples),dtype=np.float)
    
    for sample_i in range(num_samples):
        # this is the index number of the current day
        # input data will be today's index with number of days in the past
        # output data will be tommorrow's price (today_i+1)
        today_i = sample_i + number_of_days - 1
        
        input_x[sample_i,:] = prices[today_i-number_of_days+1:today_i+1]
        output_y[sample_i] = prices[today_i+1]
        
    return input_x, output_y
        
input_x, output_y = preprocess_data(test_data)

In [9]:
# look at some of the data
def print_row(i,input_x,output_y):
    print("Row %d Raw Data" % (i))
    print("-------------------")
    print("Input:")
    print(input_x[i,:])
    print("Output:")
    print(output_y[i])
    print("")
    
    print("Data Interpretation")
    print("-------------------")
    num_inputs = input_x.shape[1]
    
    print("Today's Price : %f" % (input_x[i,-1]))
    
    for xi in range(num_inputs-2,-1,-1):
        days_in_past = num_inputs - xi - 1
        print("Today - %d days: %f" % (days_in_past,input_x[i,xi]))
        
    print("Tomorrow's Price (Price to Predict): %f\n" % (output_y[i]))
    

print_row(0,input_x,output_y)
print_row(1,input_x,output_y)
print_row(-1,input_x,output_y) # last row




Row 0 Raw Data
-------------------
Input:
[ 97.314163  97.571754  97.640457  98.052658  98.378929  98.516312
  97.597534  98.421867  98.688065  97.580353]
Output:
98.79969

Data Interpretation
-------------------
Today's Price : 97.580353
Today - 1 days: 98.688065
Today - 2 days: 98.421867
Today - 3 days: 97.597534
Today - 4 days: 98.516312
Today - 5 days: 98.378929
Today - 6 days: 98.052658
Today - 7 days: 97.640457
Today - 8 days: 97.571754
Today - 9 days: 97.314163
Tomorrow's Price (Price to Predict): 98.799690

Row 1 Raw Data
-------------------
Input:
[ 97.571754  97.640457  98.052658  98.378929  98.516312  97.597534
  98.421867  98.688065  97.580353  98.79969 ]
Output:
97.795021

Data Interpretation
-------------------
Today's Price : 98.799690
Today - 1 days: 97.580353
Today - 2 days: 98.688065
Today - 3 days: 98.421867
Today - 4 days: 97.597534
Today - 5 days: 98.516312
Today - 6 days: 98.378929
Today - 7 days: 98.052658
Today - 8 days: 97.640457
Today - 9 days: 97.571754
Tomor

## Divide data into training, test, and validation sets


In [ ]:
def split_dataset(data):
    return train_data, test_data, validation_data

## Create a modular function that can be called to get data
Combine the previous functions into a simple function that can be called to get the data

In [ ]:
def get_data(symbol,train_start_date,train_end_date,
             test_start_date,test_end_date,
             validation_start_date,validation_end_date):
    return train_data, test_data, validation_data

## Create your model
Setup whatever model you plan to use for price prediction. i.e. neural network, linear regression, etc.

In [ ]:
def setup_model():
    return model

## Train your model
Define the function to train your model using the training data and the test data

In [ ]:
def train(model,train_data,test_data):
    pass

## Evaluate your model on the validation set
Return the accuracy of your model

In [ ]:
def evaluate(model,validation_data):
    pass

## Display the results
Show graphs and metrics of your model accuracy

In [ ]:
def results():
    pass

## Train and evaluate your model here
Call the functions to get the data, train the model, and evaluate it on the validation data